## Version Description:
 * In this version - we predict just the __MTR Hole Diameter__ (`R600_HD`)in TZ6 using [Bayesian Neural Network with MC DropOut](https://github.com/valyome/Neural-Networks-with-MC-Dropout/). 
 * Artificial data points synthesised from FDDN are used to train the ML model. 
 * The input features are `TZ6_Flow`,`MIXP`,`AMBP`,`AMBT` and the TZ6 CLR Restrictors.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.6g" % x)) 
from functools import reduce

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
plt.style.use('seaborn-whitegrid')

### Loading Data

In [4]:
input_features = ['TZ_Flow','MIXP','AMBP','AMBT','R610_HS1','R611_HS1','R612_HS1','R613_HS1']
usecols = ['HoV']+input_features+['R600_HD']

In [5]:
LTR_df = pd.read_csv('../../data/LTR_dataset.csv', usecols = usecols)[usecols]
LTR_df.head()

,HoV,TZ_Flow,MIXP,AMBP,AMBT,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R600_HD
0,A1,887.134249,2600.0000,101401.6000,299.386667,131,136,120,120,148
1,A2,886.764050,2600.0000,101576.3000,298.448667,131,136,120,120,149
2,A3,926.224856,2606.1928,102136.6035,297.109024,131,136,114,120,152
3,A4,915.139474,2599.8998,103195.6642,295.060027,131,136,120,120,154
4,A5,891.635528,2600.0000,102856.2000,294.755833,145,153,130,130,148


In [6]:
ASD_df = pd.read_csv('../../data/ASD_dataset.csv', usecols = usecols)[usecols]
ASD_df.head()

,HoV,TZ_Flow,MIXP,AMBP,AMBT,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R600_HD
0,ASD0,887.843325,2696.2864,101444.530,299.057324,131,136,120,120,148
1,ASD1,887.477250,2615.7850,101391.560,298.509995,131,136,120,120,148
2,ASD2,887.476875,2648.8844,101513.720,298.961758,131,136,120,120,148
3,ASD3,886.590375,2592.1955,101393.790,298.661206,131,136,120,120,148
4,ASD4,888.331625,2654.4529,101460.547,299.539189,131,136,120,120,148


In [7]:
ASD_df['R600_HD'].value_counts()

154    250
160    200
155    200
153    200
152    200
151    200
149    200
146    200
148    195
150    150
Name: R600_HD, dtype: int64

### Train BNN with FDDN (Simulation) data:

In [8]:
# Extract pandas dataframe values to numpy array
train_data = ASD_df[input_features+['R600_HD']].astype(np.float).values
test_data  = LTR_df[input_features+['R600_HD']].astype(np.float).values
print("Number of training samples:", len(train_data))
print("\nNumber of testing samples:", len(test_data))

# Separate arrays into input and output components (predictors and response variables)
x_train, y_train  = train_data[:,:-1] ,train_data[:,-1:]
x_test, y_test = test_data[:,:-1] ,test_data[:,-1:]

Number of training samples: 1995

Number of testing samples: 34


In [9]:
print('X_train Shape:',x_train.shape,'Y_Train Shape:', y_train.shape)
print('X_test Shape:',x_test.shape,'Y_Test Shape:', y_test.shape)

X_train Shape: (1995, 8) Y_Train Shape: (1995, 1)
X_test Shape: (34, 8) Y_Test Shape: (34, 1)


### Bayesian Neural Network using MC DropOut

In [10]:
import BNN_MonteCarlo_Dropout

Using TensorFlow backend.


In [11]:
# Train the BNN
bnn = BNN_MonteCarlo_Dropout.net(x_train, y_train, n_epochs=2000, n_hidden=[24,12], normalize=True)

In [12]:
# Test on ~25% of the data.
rmse, Yt_hat, MC_pred, predictive_variance = bnn.predict(x_test, y_test)

34/34 [==============================] - 0s 2ms/step
Standard rmse 0.506013
MC rmse 0.478072
test_ll -5.090436


In [13]:
print('Shape of Predictions:',Yt_hat.shape)

Shape of Predictions: (1000, 1, 34, 1)


In [14]:
Yt_hat.shape[2]

34

In [15]:
y_preds = np.zeros((Yt_hat.shape[2], Yt_hat.shape[0])) # empty array to be populated
y_means = [] # save mean for each predicted point
y_std=[] # save standard dev for each predicted
y_mins = [] 
y_maxes = []

for j in range(Yt_hat.shape[2]):
    for i in range(Yt_hat.shape[0]):
        y_preds[j][i] = Yt_hat[i][0][j] # convert the predictions into a more readable format
    y_means.append(y_preds[j].mean()) # get the mean for each prediction
    y_std.append(y_preds[j].std()) # get the standard deviation
    y_mins.append(np.amin(y_preds[j],axis = 0)) # get the min value in the array
    y_maxes.append(np.amax(y_preds[j],axis = 0)) # get the min value in the array

In [16]:
print('Length of Y_Predictions:',len(y_preds))
print('Nr. of Predictions per test point:',len(y_preds[0]))

Length of Y_Predictions: 34
Nr. of Predictions per test point: 1000


In [17]:
print(y_means)

[149.23004804992675, 149.09350512695312, 152.1688890838623, 153.6499408416748, 148.24150540161133, 148.2430943145752, 148.08336320495604, 153.01090971374512, 151.89107052612306, 149.85686563110352, 152.08136329650878, 154.01409465026856, 150.932640045166, 148.20467858886718, 148.39430781555177, 149.81582566833495, 146.60367706298828, 146.50265354919435, 148.2318542327881, 148.1554504852295, 151.98182176208496, 147.90414630126952, 154.01278147888183, 153.99906304931642, 153.90566247558593, 148.07642868041992, 148.6097225036621, 154.873043258667, 148.51768920898436, 149.94517868041993, 150.54869783020018, 159.5772512359619, 147.94968563842772, 148.7583466491699]


In [18]:
print(y_std)

[0.44914875685280814, 0.4822706925066123, 0.5328621518973174, 0.8836198023399523, 0.9051068992355373, 0.5128119423613232, 0.5800700648263027, 0.3866983523277241, 0.3908040263620649, 0.5536025184794289, 0.5824875331810033, 0.58187641950024, 0.2530882248666172, 0.5405822733973605, 0.6156731027726908, 0.3290464677002368, 1.017719649620656, 0.8468253319716377, 0.8241719980328694, 0.7758366142989814, 0.4910791097204911, 0.6272456829610338, 0.9436377197566176, 0.5490041289189431, 0.8372305639476607, 0.827615165116841, 0.6140885231927045, 0.5274896903100172, 0.55167428390906, 0.27917322381755855, 0.37565735007510054, 1.134259479194392, 0.6523904208236193, 0.5266164304855632]


In [19]:
print(y_mins)

[148.2691650390625, 148.33828735351562, 150.627685546875, 149.77906799316406, 144.8108367919922, 147.3519744873047, 147.1937255859375, 150.72604370117188, 149.7591552734375, 148.1608428955078, 150.3035888671875, 151.28282165527344, 149.79978942871094, 147.13076782226562, 147.9992218017578, 149.20358276367188, 145.59512329101562, 145.70050048828125, 146.694091796875, 145.01834106445312, 149.9029541015625, 147.34893798828125, 149.59783935546875, 151.17259216308594, 149.75918579101562, 143.92713928222656, 147.708984375, 152.39598083496094, 147.58583068847656, 149.37184143066406, 150.00550842285156, 154.00987243652344, 146.54168701171875, 148.20840454101562]


In [20]:
print(y_maxes)

[151.2535858154297, 151.78639221191406, 154.92453002929688, 157.62440490722656, 163.1629638671875, 151.57510375976562, 150.8743896484375, 154.3837432861328, 153.4854736328125, 152.5812530517578, 154.88372802734375, 158.3508758544922, 152.058349609375, 151.90171813964844, 151.4329833984375, 151.37258911132812, 151.73114013671875, 151.06863403320312, 156.46629333496094, 156.00363159179688, 153.92471313476562, 153.59664916992188, 156.61248779296875, 158.6342010498047, 154.93325805664062, 155.0478057861328, 151.4087677001953, 155.522705078125, 152.00094604492188, 151.1836700439453, 152.96119689941406, 160.9927978515625, 151.5968780517578, 151.58773803710938]


In [21]:
y_true = LTR_df['R600_HD'].values
HoV_test_data = LTR_df['HoV'].values

In [22]:
print('No. of Test HoVs:',len(HoV_test_data),'\nHoVs of Test Points:',HoV_test_data)
print('Test Points-Org Y:', y_true)

No. of Test HoVs: 34 
HoVs of Test Points: ['A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'D1' 'D2' 'E1' 'E2' 'F1' 'F2' 'F3' 'H1' 'I1' 'J1' 'M1' 'M2' 'P1' 'Q1' 'Q2' 'S1' 'S2' 'S3' 'S4' 'S5' 'T1' 'T2']
Test Points-Org Y: [148 149 152 154 148 148 148 153 152 150 152 154 151 148 148 148 146 146 148 148 152 148 154 154 154 148 148 155 148 150 150 160 148 148]


### Plot BNN Predictions with Mean and Error Bars

In [23]:
from bokeh.models import HoverTool,ColumnDataSource,Label,Range1d
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.models.markers import CircleX
output_notebook()

Loading BokehJS ...

In [24]:
# create the coordinates for the errorbars
y_xs = []
ys_std = []
y_ys = []

for x, y_mean,std,min_dia, max_dia in zip(HoV_test_data,y_means,y_std,y_mins,y_maxes):
    y_xs.append((x, x))
    ys_std.append((y_mean - std, y_mean + std))
    y_ys.append((y_mean - (y_mean - min_dia), y_mean + (max_dia - y_mean)))

In [25]:
# plot the points
f = figure(x_range=HoV_test_data, title='BNN MC DropOut - Dia Predictions with Confidence Interval', width=1400)
dot_hover_tooltips = HoverTool(names=['DiaPoints'],tooltips=[("HoV","@x"),("Dia", "@y mm")])
whisker_hover_tooltips = HoverTool(names=['Whiskers'],tooltips=[("Dia", "@y mm")])
multiline_hover_tooltips = HoverTool(names=['LinePlot'],tooltips=[("Dia", "$y mm")])

f.add_tools(dot_hover_tooltips,whisker_hover_tooltips,multiline_hover_tooltips)

f.xaxis.axis_label = 'Head of Versions'
f.yaxis.axis_label = 'Diameter Predictions (mm)'

f.multi_line(y_xs, ys_std, color='blue', line_width = 3, legend = 'Mean Prediction +/- Std.Dev')
f.multi_line(y_xs, y_ys, color='deepskyblue',name='LinePlot',legend = 'Mean Prediction +/- Min & Max Value')
f.diamond(HoV_test_data, y_true, color='red', size=12, line_alpha=0,name='DiaPoints', legend = 'True Dia')
f.circle(HoV_test_data, y_means, color='blue', size=9, line_alpha=0,name='DiaPoints', legend = 'Predicted Dia (Mean)')

# whiskers (almost-0 height rects simpler than segments)
for i in range(len(y_xs)):
    f.rect(y_xs[i], y_ys[i], 0.2, 0.001, line_color="deepskyblue",name='Whiskers')

f.y_range=Range1d(135, 165)
f.legend.location = "top_center"
f.legend.click_policy="hide"

output_file('plots/BNN_MCDropOut_v3_ASD_LTR_Predictions.html', mode='inline')

In [26]:
show(f)